In [ ]:
import numpy as np
from faster_whisper import WhisperModel
from yt_dlp import YoutubeDL
import io

In [10]:
# Configuration
model_size = "small"  # You can choose "tiny", "base", "small", "medium", "large"
youtube_url = "https://www.youtube.com/watch?v=SKA88JKFVdk"  # Replace with your livestream URL
sample_rate = 16000  # Whisper models expect 16kHz audio
chunk_duration = 10  # Duration of each audio chunk in seconds
model = WhisperModel(model_size, device="cpu", compute_type="int8")  # Use "cuda" for GPU

In [17]:
# Function to download and stream audio from YouTube livestream
def stream_youtube_audio(url):
    ydl_opts = {
        'format': 'bestaudio/best',
        'extractaudio': True,
        'audioformat': 'wav',
        'noplaylist': True,
        'quiet': True,
        'live_from_start': True,  # Ensure live stream is captured from the start
    }
    with YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=False)
        audio_url = info['url']
        return audio_url

In [13]:
# Function to transcribe audio chunks
def transcribe_audio_chunk(audio_chunk):
    # Convert audio chunk to numpy array
    audio_data = np.frombuffer(audio_chunk.raw_data, dtype=np.int16).astype(np.float32) / 32768.0
    
    # Transcribe the audio
    segments, info = model.transcribe(audio_data, beam_size=5)
    
    # Print the transcription
    for segment in segments:
        print(f"[{segment.start:.2f}s -> {segment.end:.2f}s] {segment.text}")

In [14]:
# Main function
def main():
    # Get the audio stream URL
    audio_stream_url = stream_youtube_audio(youtube_url)
    
    # Use pydub to stream the audio in chunks
    stream = AudioSegment.from_file(audio_stream_url, format="wav")
    chunk_length = chunk_duration * 1000  # Convert to milliseconds
    
    for i in range(0, len(stream), chunk_length):
        chunk = stream[i:i + chunk_length]
        
        # Transcribe the chunk
        transcribe_audio_chunk(chunk)
        
        # Optional: Play the chunk (for debugging)
        # play(chunk)

In [18]:
if __name__ == "__main__":
    main()

FileNotFoundError: [Errno 2] No such file or directory: 'https://manifest.googlevideo.com/api/manifest/dash/expire/1739939000/ei/WAi1Z470N97i9fwPwdjauQ0/ip/103.232.219.159/id/SKA88JKFVdk.2/source/yt_live_broadcast/requiressl/yes/xpc/EgVo2aDSNQ%3D%3D/tx/51388542/txs/51388542%2C51388543%2C51388544%2C51388545/hfr/all/as/fmp4_audio_clear%2Cfmp4_sd_hd_clear/defrag/1/spc/RjZbSW6WSu9nhZjplMDu9zgMIHJy8aQoyN98rj3q0CBtyXki24nHtYWbHaxwU0g/vprv/1/rqh/2/pacing/0/keepalive/yes/fexp/51326932/itag/0/playlist_type/LIVE/sparams/expire%2Cei%2Cip%2Cid%2Csource%2Crequiressl%2Cxpc%2Ctx%2Ctxs%2Chfr%2Cas%2Cdefrag%2Cspc%2Cvprv%2Crqh%2Citag%2Cplaylist_type/sig/AJfQdSswRQIgCdSh3wFQXrQ97D_GWeYGuPeIe4m0axhQ0y4JnJq-yGQCIQCvDDF6DutstIsbrjBe1JILN67TQjxDWjnPr7T1_GfSfA%3D%3D'